In [4]:
#import sys
#sys.path.append('my/path/to/module/folder')
%matplotlib inline

import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import scipy.stats
from scipy.stats import pearsonr

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC 

# !pip install netCDF4

from tqdm import tqdm 
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [8]:
# Modified code originally provided by Ankur Mahesh from ClimateAi.
def load_enso_indices():
  """
  Reads in the txt data file to output a pandas Series of ENSO vals

  outputs
  -------

    pd.Series : monthly ENSO values starting from 1980-01-01
  """

  with open(anomaly_path) as f:
    line = f.readline()
    enso_vals = []
    while line:
        yearly_enso_vals = map(float, line.split()[1:]) 
        enso_vals.extend(yearly_enso_vals)
        line = f.readline()

  enso_vals = pd.Series(enso_vals)
  enso_vals.index = pd.date_range('1980-01-01',freq='MS',
                                  periods=len(enso_vals))
  enso_vals.index = pd.to_datetime(enso_vals.index)
  return enso_vals

def assemble_basic_predictors_predictands(start_date, end_date, lead_time,
                                    use_pca=False, n_components=32):
  """
  inputs
  ------

      start_date        str : the start date from which to extract sst
      end_date          str : the end date 
      lead_time         str : the number of months between each sst
                              value and the target Nino3.4 Index
      use_pca          bool : whether or not to apply principal components
                              analysis to the sst field
      n_components      int : the number of components to use for PCA

  outputs
  -------
      Returns a tuple of the predictors (np array of sst temperature anomalies) 
      and the predictands (np array the ENSO index at the specified lead time).

  """
  ds = xr.open_dataset(godas_path)
  sst = ds['deepTemp'].sel(time=slice(start_date, end_date)) 
  num_time_steps = sst.shape[0]

  sst = sst.values.reshape(num_time_steps, -1)
  sst[np.isnan(sst)] = 0

  if use_pca:
    pca = sklearn.decomposition.PCA(n_components=n_components)
    pca.fit(sst)
    X = pca.transform(sst)
  else:
    X = sst

  start_date_plus_lead = pd.to_datetime(start_date) + \
                        pd.DateOffset(months=lead_time)
  end_date_plus_lead = pd.to_datetime(end_date) + \
                      pd.DateOffset(months=lead_time)
  y = load_enso_indices()[slice(start_date_plus_lead, 
                                end_date_plus_lead)]


  ds.close()
  return X, y

def plot_nino_time_series(y, predictions, depth, month_lead, title):
  """
  inputs
  ------
    y           pd.Series : time series of the true Nino index
    predictions np.array  : time series of the predicted Nino index (same
                            length and time as y)
    title                 : the title of the plot
    depth                 : depth level being considered
    month_lead            : month lead time 

  outputs
  -------
    None.  Displays the plot
  """
  images_dir = '/cw3e/mead/projects/cdd101/Figures/NOAA_GODAS_Results'
  figName_png = str(depth)+'m_Predictions_'+str(month_lead)+'month.png'
  figName_pdf = str(depth)+'m_Predictions_'+str(month_lead)+'month.pdf'

  predictions = pd.Series(predictions, index=y.index)
  predictions = predictions.sort_index()
  y = y.sort_index()

  plt.plot(y, label='Ground Truth')
  plt.plot(predictions, '--', label='ML Predictions')
  plt.legend(loc='best')
  plt.title(title)
  plt.ylabel(str(depth)+ " " + 'm Temperature Anomalies')
  plt.xlabel('Date')
  plt.savefig(images_dir+"/"+figName_png, bbox_inches='tight')
  plt.savefig(images_dir+"/"+figName_pdf, bbox_inches='tight')
  plt.close()
    


In [9]:
depth_arr = np.array([105, 155, 205])
month_lead_arr = np.array([2, 6, 12])

for mm in range(len(month_lead_arr)):
  for ii in range(len(depth_arr)):
    depth_sel = depth_arr[ii]
    month_lead = month_lead_arr[mm]
    
    depth_string = str(depth_sel)
    godas_path = '/cw3e/mead/projects/cdd101/Files/GODAS/godasData_'+depth_string+'m.nc'
    anomaly_path = '/cw3e/mead/projects/cdd101/Files/GODAS/fixed_deepTemp_anomalies_'+depth_string+'m.txt'
    # Select godas and anomaly path

    X_train, y_train = assemble_basic_predictors_predictands('1980-01-01','1995-12-31', lead_time=month_lead)                                                         
    X_val, y_val = assemble_basic_predictors_predictands('1997-01-01','2002-12-31', lead_time=month_lead)

    # Linear Regression
    regr = sklearn.linear_model.LinearRegression()
    regr.fit(X_train,y_train)
    predictions = regr.predict(X_val)
    corr, _ = scipy.stats.pearsonr(predictions, y_val)
    plot_nino_time_series(y_val, predictions, depth_sel, month_lead,
        'Predicted and True Ocean Temperature Anomalies at'+' '+str(month_lead)+' '+'Month Lead Time. \n Corr: {:.2f}'.format(corr))

Use GODAS as predictor data with COBE anomalies as predictands

In [6]:
# Modified code originally provided by Ankur Mahesh from ClimateAi.
def load_enso_indices():
  """
  Reads in the txt data file to output a pandas Series of ENSO vals

  outputs
  -------

    pd.Series : monthly ENSO values starting from 1980-01-01
  """

  with open(anomaly_path) as f:
    line = f.readline()
    enso_vals = []
    while line:
        yearly_enso_vals = map(float, line.split()[1:]) 
        enso_vals.extend(yearly_enso_vals)
        line = f.readline()

  enso_vals = pd.Series(enso_vals)
  enso_vals.index = pd.date_range('1980-01-01',freq='MS',
                                  periods=len(enso_vals))
  enso_vals.index = pd.to_datetime(enso_vals.index)
  return enso_vals

def assemble_basic_predictors_predictands(start_date, end_date, lead_time,
                                    use_pca=False, n_components=32):
  """
  inputs
  ------

      start_date        str : the start date from which to extract sst
      end_date          str : the end date 
      lead_time         str : the number of months between each sst
                              value and the target Nino3.4 Index
      use_pca          bool : whether or not to apply principal components
                              analysis to the sst field
      n_components      int : the number of components to use for PCA

  outputs
  -------
      Returns a tuple of the predictors (np array of sst temperature anomalies) 
      and the predictands (np array the ENSO index at the specified lead time).

  """
  ds = xr.open_dataset(godas_path)
  sst = ds['deepTemp'].sel(time=slice(start_date, end_date)) 
  num_time_steps = sst.shape[0]

  sst = sst.values.reshape(num_time_steps, -1)
  sst[np.isnan(sst)] = 0

  if use_pca:
    pca = sklearn.decomposition.PCA(n_components=n_components)
    pca.fit(sst)
    X = pca.transform(sst)
  else:
    X = sst

  start_date_plus_lead = pd.to_datetime(start_date) + \
                        pd.DateOffset(months=lead_time)
  end_date_plus_lead = pd.to_datetime(end_date) + \
                      pd.DateOffset(months=lead_time)
  y = load_enso_indices()[slice(start_date_plus_lead, 
                                end_date_plus_lead)]


  ds.close()
  return X, y

def plot_nino_time_series(y, predictions, depth, month_lead, title):
  """
  inputs
  ------
    y           pd.Series : time series of the true Nino index
    predictions np.array  : time series of the predicted Nino index (same
                            length and time as y)
    title                 : the title of the plot
    depth                 : depth level being considered
    month_lead            : month lead time 

  outputs
  -------
    None.  Displays the plot
  """
  images_dir = '/cw3e/mead/projects/cdd101/Figures/NOAA_GODAS_Results/SST_Anomalies'
  figName_png = str(depth)+'m_Predictions_'+str(month_lead)+'month.png'
  figName_pdf = str(depth)+'m_Predictions_'+str(month_lead)+'month.pdf'

  predictions = pd.Series(predictions, index=y.index)
  predictions = predictions.sort_index()
  y = y.sort_index()

  plt.plot(y, label='Ground Truth')
  plt.plot(predictions, '--', label='ML Predictions')
  plt.legend(loc='best')
  plt.title(title)
  plt.ylabel('SST Anomalies Using' + " " + str(depth) + "m Depth Data")
  plt.xlabel('Date')
  plt.savefig(images_dir+"/"+figName_png, bbox_inches='tight')
  plt.savefig(images_dir+"/"+figName_pdf, bbox_inches='tight')
  plt.close()

In [7]:
depth_arr = np.array([105, 155, 205])
month_lead_arr = np.array([2, 6, 12])
anomaly_path = '/cw3e/mead/projects/cdd101/Files/nino34.long.anom.data.trimmed.txt'
    
for mm in range(len(month_lead_arr)):
  for ii in range(len(depth_arr)):
    depth_sel = depth_arr[ii]
    month_lead = month_lead_arr[mm]
    
    depth_string = str(depth_sel)
    godas_path = '/cw3e/mead/projects/cdd101/Files/GODAS/godasData_'+depth_string+'m.nc'

    X_train, y_train = assemble_basic_predictors_predictands('1980-01-01','1995-12-31', lead_time=month_lead)                                                         
    X_val, y_val = assemble_basic_predictors_predictands('1997-01-01','2002-12-31', lead_time=month_lead)

    # Linear Regression
    regr = sklearn.linear_model.LinearRegression()
    regr.fit(X_train,y_train)
    predictions = regr.predict(X_val)
    corr, _ = scipy.stats.pearsonr(predictions, y_val)
    plot_nino_time_series(y_val, predictions, depth_sel, month_lead,
        'Predicted and True Ocean Temperature Anomalies at'+' '+str(month_lead)+' '+'Month Lead Time. \n Corr: {:.2f}'.format(corr))